In [1]:
## use kernel "napari_spatialdata"
# here we import all needed libraries
import spatialdata as sd
from napari_spatialdata import Interactive
import pandas as pd
from ome_zarr.io import parse_url
from ome_zarr.reader import Node, Reader
import numpy as np
import tifffile as tif
from tifffile import TiffFile

In [3]:
## declare paths
path_to_decoded_csv = '/nfs/t233_imaging/NT_SNP/playground_Tong/Pool3iMGL_10rsID_multianchor/work/eb/aa1119a052897c486c1909c1324368/Pool3iMGL_10rsID_decoded_spots.csv'
path_to_spatial_data = '/nfs/t233_imaging/NT_SNP/playground_Tong/Pool3iMGL_10rsID_multianchor/output/spatialdata/Pool3iMGL_10rsID.sdata/'

## 1. Load spatial data and display it

In [10]:
sdata = sd.read_zarr(path_to_spatial_data)
interactive = Interactive(sdata)
viewer = interactive._viewer

## when you open different layers change belnding to "additive" to see them all together overlayed

2024-10-02 16:46:40.896 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
/opt/conda/envs/napari_spatialdata/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2024-10-02 16:46:41.426 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:46:41.430 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:46:41.431 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:46:53.643 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:46:53.648 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.


## 2a. Prepare all layers of decoded spot types

In [11]:
decoded_peaks = pd.read_csv(path_to_decoded_csv, sep=",")
decoded_group = decoded_peaks.groupby("Name")
decoded_peaks


,Name,Code,Probability,y_int,x_int
0,rs13296_G,GTGTTGTGTTTT,0.999989,1349,14
1,rs1054411_C,GGGGTTTTTTTT,0.999978,1538,116
2,rs1054411_C,GGGGTTTTTTTT,0.999959,1547,116
3,rs1054411_C,GGGGTTTTTTTT,0.999976,1915,219
4,rs14867_C,GTTGTTTTGTTG,0.994291,1955,189
...,...,...,...,...,...
1839250,background,0000,0.999760,20289,20264
1839251,background,0000,0.999760,20296,20212
1839252,background,0000,0.999760,20298,20264
1839253,background,0000,0.999760,20302,20271


2024-10-02 16:52:04.448 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.


## 2b. Upload all layers of decoded spot types to napari

In [14]:
spot_layers = {}
target_genes = decoded_peaks.Name.unique()
for n, grp in decoded_group:
    if n in target_genes:
        random_color = np.array([np.random.rand(),np.random.rand(),np.random.rand()])
        spot_layers[n] = viewer.add_points(grp[["y_int", "x_int"]].values,
                                           name=n, visible=False,
                                           size=5,  ## SIZE OF THE SPOT - FEEL FREE TO CHANGE
                                           face_color=random_color,
                                           edge_color=random_color,
                                           scale=[1, 1],
                                           blending="additive"
                                          )

2024-10-02 16:53:05.734 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:05.870 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:05.874 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:06.069 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:06.074 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:06.240 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:06.244 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:09.865 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:09.869 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:53:10.254 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updati

## 3. Highlight the cell with a specific id

In [17]:
cell_id = 666 

label_image = sdata.labels['cell_labels']['scale0'].image
label_one_cell = np.zeros_like(label_image); label_one_cell = label_one_cell.astype('uint8')
label_one_cell[label_image==cell_id] = 255
viewer.add_image(label_one_cell, name = str('Cell ' + str(cell_id)), blending = 'additive')

/opt/conda/envs/napari_spatialdata/lib/python3.9/site-packages/napari/_vispy/layers/image.py:251: UserWarning: data shape (20396, 20396) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/opt/conda/envs/napari_spatialdata/lib/python3.9/site-packages/napari/_vispy/layers/image.py:251: UserWarning: data shape (20396, 20396) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
2024-10-02 16:55:36.895 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-10-02 16:55:36.900 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.


<Image layer 'Cell 666' at 0x7fd4a7e04eb0>

/opt/conda/envs/napari_spatialdata/lib/python3.9/site-packages/napari/_vispy/layers/image.py:251: UserWarning: data shape (20396, 20396) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/opt/conda/envs/napari_spatialdata/lib/python3.9/site-packages/napari/_vispy/layers/image.py:251: UserWarning: data shape (20396, 20396) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
